# Dataset generation

## 1. Import

In [8]:
import numpy as np
import networkx as nx
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

## 2. Generating Distance Matrices

In [16]:
N = 1000    # number of samples
M = 50      # Number of nodes

X = np.random.uniform(0,1, size=(N, M, M))

X = np.triu(X)
for i in range(X.shape[0]):
    X[i,:] = X[i,:] + X[i,:].T
    X[i,:] = X[i,:] - np.diag(np.diag(X[i,:]))
print(X[0])

[[0.         0.72106466 0.24364316 ... 0.20698357 0.88329681 0.68969389]
 [0.72106466 0.         0.85068031 ... 0.20303993 0.18380799 0.46225688]
 [0.24364316 0.85068031 0.         ... 0.1233658  0.00455358 0.04200989]
 ...
 [0.20698357 0.20303993 0.1233658  ... 0.         0.89413285 0.24094281]
 [0.88329681 0.18380799 0.00455358 ... 0.89413285 0.         0.42522859]
 [0.68969389 0.46225688 0.04200989 ... 0.24094281 0.42522859 0.        ]]


In [17]:
#X = X.reshape((N,M*M))

## 3. Building Solutions

In [18]:
"""Simple Travelling Salesperson Problem (TSP) between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = X[0]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)

main()

Objective: 0 miles
Route for vehicle 0:
 0 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 37 -> 36 -> 35 -> 34 -> 33 -> 32 -> 31 -> 30 -> 29 -> 28 -> 27 -> 26 -> 25 -> 24 -> 23 -> 22 -> 21 -> 20 -> 19 -> 18 -> 17 -> 16 -> 15 -> 14 -> 13 -> 12 -> 11 -> 10 -> 9 -> 8 -> 7 -> 6 -> 5 -> 4 -> 3 -> 2 -> 1 -> 0

